In [ ]:
!pip install keras

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

In [ ]:
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"/content/indic_nlp_resources"

In [ ]:
!pip install fasttext

In [ ]:
!wget https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/embedding-v2/indicnlp.ft.hi.300.bin

In [ ]:
import fasttext
embedding_model = fasttext.load_model('/content/indicnlp.ft.hi.300.bin')

In [ ]:
cd /content/calling-out-bluff/Model3(SkipFlow)/

In [ ]:
import  keras.layers  as  klayers 
from keras.preprocessing.text import text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Input, Embedding, GlobalAveragePooling1D, Concatenate, Activation, Lambda, BatchNormalization, Convolution1D, Dropout
from keras.preprocessing.text import Tokenizer
import numpy as np
import nltk
from quadratic_weighted_kappa import QWK
from sklearn.metrics import cohen_kappa_score
from keras.models import Model
from keras import backend as K
from keras.layers import Layer, InputSpec
from keras.optimizers import adam_v2
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import initializers
from scipy import stats

In [ ]:
class Neural_Tensor_layer(Layer):
	def __init__(self,output_dim,input_dim=None, **kwargs):
		self.output_dim=output_dim
		self.input_dim=input_dim
		if self.input_dim:
			kwargs['input_shape']=(self.input_dim,)
# 		print("YAYY", input_dim, output_dim)
		super(Neural_Tensor_layer,self).__init__(**kwargs)

	def call(self,inputs,mask=None):
		e1=inputs[0]
		e2=inputs[1]
		batch_size=K.shape(e1)[0]
		k=self.output_dim
		

		feed_forward=K.dot(K.concatenate([e1,e2]),self.V)

		bilinear_tensor_products = [ K.sum((e2 * K.dot(e1, self.W[0])) + self.b, axis=1) ]

		for i in range(k)[1:]:	
			btp=K.sum((e2*K.dot(e1,self.W[i]))+self.b,axis=1)
			bilinear_tensor_products.append(btp)

		result=K.tanh(K.reshape(K.concatenate(bilinear_tensor_products,axis=0),(batch_size,k))+feed_forward)

		return result
    
	def build(self,input_shape):
		mean=0.0
		std=1.0
		k=self.output_dim
		d=self.input_dim
		##truncnorm generate continuous random numbers in given range
		W_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(k,d,d))
		V_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(2*d,k))
		self.W=K.variable(W_val)
		self.V=K.variable(V_val)
		self.b=K.zeros((self.input_dim,))
		self._trainable_weights=[self.W,self.V,self.b]    

	def compute_output_shape(self, input_shape):
		batch_size=input_shape[0][0]
		return(batch_size,self.output_dim)

In [ ]:
class Temporal_Mean_Pooling(Layer): # conversion from (samples,timesteps,features) to (samples,features)
	def __init__(self, **kwargs):
		super(Temporal_Mean_Pooling,self).__init__(**kwargs)
		# masked values in x (number_of_samples,time)
		self.supports_masking=True
		# Specifies number of dimensions to each layer
		self.input_spec=InputSpec(ndim=3)
        
	def call(self,x,mask=None):
		if mask is None:
			mask=K.mean(K.ones_like(x),axis=-1)

		mask=K.cast(mask,K.floatx())
				#dimension size single vec/number of samples
		return K.sum(x,axis=-2)/K.sum(mask,axis=-1,keepdims=True)        

	def compute_mask(self,input,mask):
		return None
    
    
	def compute_output_shape(self,input_shape):
		return (input_shape[0],input_shape[2])

In [ ]:
EMBEDDING_DIM=300
MAX_NB_WORDS=4000

MAX_SEQUENCE_LENGTH=500
VALIDATION_SPLIT=0.20
DELTA=20

texts=[]

labels=[]


originals = []


In [ ]:
essay_type = '2'

fp=open("dataset.tsv",'r', encoding="utf-8", errors="ignore")
fp.readline()
originals = []

In [ ]:
for line in fp:
    temp=line.split("\t")
    if(temp[3]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        originals.append(float(temp[5]))
# print(originals)
fp.close()
# print(originals)
print("range min - ", min(originals) , " ; range max - ", max(originals))


In [ ]:
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))

In [ ]:
from indicnlp.tokenize import indic_tokenize  

range_min = min(originals)
range_max = max(originals)

fp=open("dataset.tsv",'r', encoding="utf-8", errors="ignore")
fp.readline()
sentences=[]
for line in fp:
    temp=line.split("\t")
    if(temp[3]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        texts.append(temp[6])
        labels.append((float(temp[5])-range_min)/(range_max-range_min)) ## why ??  - normalize to range [0-1]
        line=temp[6].strip()
        sentences.append(indic_tokenize.trivial_tokenize(line))

fp.close()


In [ ]:
#!pip install torch

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModel.from_pretrained("ai4bharat/indic-bert")
input_ids = torch.tensor(tokenizer.encode(sen[:128])).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple

In [ ]:
last_hidden_states

In [ ]:
labels
print(len(labels))

In [ ]:
texts[1]

In [ ]:
textso = texts.copy()

In [ ]:
print("text labels appended %s" %len(texts))

labels=np.asarray(labels)
print(labels)
print(len(labels))

In [ ]:
fast_emb = {}
for i in sentences:
  temp1 = np.zeros((1, EMBEDDING_DIM))
  for w in i:
    #print(w,np.asarray([float(j) for j in embedding_model[w]]))
    fast_emb[w] = embedding_model[w]

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')

In [ ]:
tokenized = tokenizer(texts)['input_ids']

In [ ]:
sequences = []
for i in tokenized:
  sequences.append(list(map(lambda k:[k], i)))

In [ ]:
word_index = tokenizer.vocab

In [ ]:
print('Found %s unique tokens.' % len(word_index))

In [ ]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) #padding to max_length
print('Shape of data tensor:', data.shape)

In [ ]:
data = np.reshape(data,(1600,500))

In [ ]:
indices=np.arange(data.shape[0]) #with one argument, start=0, step =1
print(data.shape)
np.random.shuffle(indices)
data=data[indices]
print(data.shape)
labels=labels[indices]
# np.reshape(labels, ())
print(labels.shape)
validation_size=int(VALIDATION_SPLIT*data.shape[0])
print(validation_size)

In [ ]:
x_train=data[:-validation_size] #data-validation data
print(x_train.shape)
# print(x_train)
# print(labels)
y_train=labels[:-validation_size]
# print(y_train.transpose)
print(y_train.shape)
# y_train = np.reshape(y_train, (1427, 1))
# print(y_train_new)
# print(y_train)
x_val=data[-validation_size:]
print(x_val.shape)
y_val=labels[-validation_size:]

In [ ]:
embedding_matrix = np.zeros((len(word_index), EMBEDDING_DIM))
print(embedding_matrix.shape)

In [ ]:
for word,i in word_index.items():
	if(i>=len(word_index)):
		continue
	if word in fast_emb:
			embedding_matrix[i]=fast_emb[word]
vocab_size=len(word_index)
print(vocab_size)

In [ ]:
for i in embedding_matrix:
  if np.all(i != 0):
    print(i)
    break

In [ ]:
count = 0
for word,i in word_index.items():
  if(i>=len(word_index)):
    continue
  if word in fast_emb:
    count = count + 1
print(count)

In [ ]:
embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=True,
							trainable=False)
# print(embedding_layer.shape)
side_embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=False,
							trainable=False)

In [ ]:
def SKIPFLOW(lstm_dim, lr, lr_decay, k, eta, delta, activation, maxlen=MAX_SEQUENCE_LENGTH, seed=None):
    e = Input(name='essay',shape=(maxlen,))
    print("e", e)
#     trad_feats=Input(shape=(7,))
#     print("trad_feats", trad_feats)
    embed = embedding_layer(e)
    print(embed.shape)
    lstm_layer=LSTM(lstm_dim,return_sequences=True)
    print(lstm_layer)
    hidden_states=lstm_layer(embed)
    htm=Temporal_Mean_Pooling()(hidden_states)    
    side_embed = side_embedding_layer(e)
    side_hidden_states=lstm_layer(side_embed)    
    tensor_layer=Neural_Tensor_layer(output_dim=k,input_dim=lstm_dim)
#     print(input_dim, output_dim)
    pairs = [((eta + i * delta) % maxlen, (eta + i * delta + delta) % maxlen) for i in range(maxlen // delta)]
    hidden_pairs = [ (Lambda(lambda t: t[:, p[0], :])(side_hidden_states), Lambda(lambda t: t[:, p[1], :])(side_hidden_states)) for p in pairs]
    sigmoid = Dense(1, activation="sigmoid", kernel_initializer=initializers.glorot_normal(seed=seed))
    coherence = [sigmoid(tensor_layer([hp[0], hp[1]])) for hp in hidden_pairs]
    co_tm=Concatenate()(coherence[:]+[htm])
    dense = Dense(256, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(co_tm)
    dense = Dense(128, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
    dense = Dense(64, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
    out = Dense(1, activation="sigmoid")(dense)
    smodel = Model(inputs=[e], outputs=[out])
    print("input", [e])
    print("outputs", out)
    adam = Adam(lr=lr, decay=lr_decay)
    smodel.compile(loss="mean_squared_error", optimizer='adam', metrics=["MSE"])
    return smodel

In [ ]:
from tensorflow.keras.optimizers import Adam
# from keras.utils.vis_utils import plot_model
earlystopping = EarlyStopping(monitor="val_MSE", patience=5)
sf_2 = SKIPFLOW(lstm_dim=500, lr=0.01, lr_decay=0, k=2, eta=13, delta=50, activation="relu", seed=None)
sf_2.summary()
# plot_model(sf_1)

In [ ]:
# print(sf_1)
epochs = 8
# epochs = 1000
print(type(x_train))
# y_train = np.asarray(y_train)
print(type(y_train))

sf_2.fit(x_train, y_train, batch_size=64, epochs=epochs, validation_data=([x_val], y_val), callbacks=[earlystopping])

In [ ]:
y_pred=sf_2.predict([x_val])
y_pred

In [ ]:
y_val_fin = [int(round(a*(range_max-range_min)+range_min)) for a in y_val]
print(y_val_fin)

In [ ]:
y_pred_fin =[int(round(a*(range_max-range_min)+range_min)) for a in y_pred.reshape(320).tolist()]
print(y_pred_fin)

In [ ]:
print(cohen_kappa_score(y_val_fin,y_pred_fin,weights="quadratic"))

In [ ]:
def Cmatrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def QWK_new(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [ ]:
QWK_new(y_val_fin, y_pred_fin)

In [ ]:

# from keras.utils.vis_utils import plot_model
earlystopping = EarlyStopping(monitor="val_MSE", patience=5)
sf_2 = SKIPFLOW(lstm_dim=700, lr=0.01, lr_decay=0, k=2, eta=13, delta=50, activation="relu", seed=None)
sf_2.summary()
# plot_model(sf_1)